### Enviroment used : jupyter lab
### Solution runs on colab

## Installing the package

In [67]:
!pip install MultiLabel-Transformer

In [68]:
!pip install lightgbm==3.1.1 

## Importing the packages

In [69]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
import warnings
warnings.filterwarnings('ignore')
# from xgboost import XGBClassifier
from MLE_Trans import MLETransformer
from sklearn.ensemble import IsolationForest
from imblearn.over_sampling import SMOTE,RandomOverSampler

In [70]:
## Reading the dataset
train = pd.read_csv('../input/indabax-ghana-avic-hack-ds/files/Train.csv',parse_dates=['Policy Start Date','Policy End Date','First Transaction Date'])
test = pd.read_csv('../input/indabax-ghana-avic-hack-ds/files/Test.csv',parse_dates=['Policy Start Date','Policy End Date','First Transaction Date'])
ss = pd.read_csv('../input/indabax-ghana-avic-hack-ds/files/SampleSubmission.csv')
#
train.head()

,ID,Policy Start Date,Policy End Date,Gender,Age,First Transaction Date,No_Pol,Car_Category,Subject_Car_Colour,Subject_Car_Make,LGA_Name,State,ProductName,target
0,ID_0040R73,2010-05-14,2011-05-13,Male,30,2010-05-14,1,Saloon,Black,TOYOTA,NaN,NaN,Car Classic,0
1,ID_0046BNK,2010-11-29,2011-11-28,Female,79,2010-11-29,1,JEEP,Grey,TOYOTA,NaN,NaN,Car Classic,1
2,ID_005QMC3,2010-03-21,2011-03-20,Male,43,2010-03-21,1,Saloon,Red,TOYOTA,NaN,NaN,Car Classic,0
3,ID_0079OHW,2010-08-21,2011-08-20,Male,2,2010-08-21,1,NaN,NaN,NaN,NaN,NaN,CarSafe,0
4,ID_00BRP63,2010-08-29,2010-12-31,Entity,20,2010-08-29,3,NaN,NaN,NaN,Lagos,Lagos,Muuve,1


## Exploratory Data Analysis

In [71]:
train.State.value_counts()

Lagos              3186
Benue               626
Abuja-Municipal     232
Eti-Osa             217
Ibeju-Lekki         123
                   ... 
Bayelsa               1
Akoko-West            1
Awka-North            1
Ndokwa-East           1
Isoko-south           1
Name: State, Length: 111, dtype: int64

In [72]:
train.loc[train.State =='Benue']

,ID,Policy Start Date,Policy End Date,Gender,Age,First Transaction Date,No_Pol,Car_Category,Subject_Car_Colour,Subject_Car_Make,LGA_Name,State,ProductName,target
8,ID_00N7XT1,2010-07-06,2011-07-05,Female,71,2010-07-06,2,JEEP,Black,REXTON,Badagry,Benue,Car Classic,0
26,ID_02DJWEN,2010-01-23,2011-01-16,Male,41,2010-01-23,1,Saloon,Silver,TOYOTA,Yaba,Benue,Car Classic,0
29,ID_02O5FDM,2010-02-10,2011-02-09,Joint Gender,20,2010-02-10,1,NaN,As Attached,TOYOTA,Aguda,Benue,Car Classic,0
46,ID_04JEN82,2010-03-08,2011-02-09,Male,41,2010-03-08,2,Saloon,Silver,TOYOTA,IbadanCentral,Benue,Car Plus,0
56,ID_05FC0AD,2010-07-04,2011-07-03,Female,50,2010-07-04,1,Saloon,NaN,TOYOTA,OBANIKORO,Benue,Car Classic,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11993,ID_ZR94K40,2010-03-28,2011-03-27,Male,86,2010-03-28,1,NaN,NaN,TOYOTA,Kosofe,Benue,Car Classic,0
12004,ID_ZSEUUD8,2010-03-03,2011-03-01,Male,56,2010-03-03,1,Saloon,NaN,TOYOTA,Oredo,Benue,Car Classic,0
12009,ID_ZSQB2BN,2010-03-20,2011-01-27,Male,46,2010-03-20,2,Saloon,Black,Volkswagen,Katagum,Benue,Car Classic,0
12020,ID_ZTSQ90V,2010-02-06,2011-02-05,Male,57,2010-02-06,1,Saloon,Silver,TOYOTA,Egbeda,Benue,Car Classic,0


In [73]:
train.describe()

,Age,No_Pol,target
count,12079.000000,12079.000000,12079.000000
mean,42.234539,1.307227,0.120457
std,97.492565,0.733085,0.325509
min,-6099.000000,1.000000,0.000000
25%,35.000000,1.000000,0.000000
50%,41.000000,1.000000,0.000000
75%,50.000000,1.000000,0.000000
max,320.000000,10.000000,1.000000


## Exploratory data analysis

In [74]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12079 entries, 0 to 12078
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   ID                      12079 non-null  object        
 1   Policy Start Date       12079 non-null  datetime64[ns]
 2   Policy End Date         12079 non-null  datetime64[ns]
 3   Gender                  11720 non-null  object        
 4   Age                     12079 non-null  int64         
 5   First Transaction Date  12079 non-null  datetime64[ns]
 6   No_Pol                  12079 non-null  int64         
 7   Car_Category            8341 non-null   object        
 8   Subject_Car_Colour      5117 non-null   object        
 9   Subject_Car_Make        9603 non-null   object        
 10  LGA_Name                5603 non-null   object        
 11  State                   5591 non-null   object        
 12  ProductName             12079 non-null  object

In [75]:
train.shape

(12079, 14)

In [76]:
test.shape

(1202, 13)

## Dropping some columns

In [77]:
y=train['target'] # target columns
train = train.drop(['target','ID','Subject_Car_Colour','First Transaction Date'],axis=1)
test = test.drop(['ID','Subject_Car_Colour','First Transaction Date'],axis=1)

### Concatenation of both train and test dataset

In [78]:
train['source'] ='train'
test['source'] ='test' # creating a new column to distinguish between train and test

df =pd.concat([train,test],ignore_index=True,sort=False)
print('train shape',train.shape)
print('test shape',test.shape)
print('data shape',df.shape)

train shape (12079, 11)
test shape (1202, 11)
data shape (13281, 11)


## Data cleaning and Feature extraction

#### Dealing with outliers in the age column by replacing them with null value

In [79]:
df['Age'] = np.where(df['Age'].between(-6099,1), np.NaN, df['Age'])
df['Age'] = np.where(df['Age'].between(90,320), np.NaN, df['Age'])

In [80]:
# df['Age'] = df.Age.fillna(-9999)

In [81]:
df.describe()# checking the effect of the cleaning

,Age,No_Pol
count,12570.0000,13281.000000
mean,41.9035,1.302763
std,12.8081,0.723195
min,2.0000,1.000000
25%,35.0000,1.000000
50%,40.0000,1.000000
75%,49.0000,1.000000
max,89.0000,10.000000


In [82]:
df.columns

Index(['Policy Start Date', 'Policy End Date', 'Gender', 'Age', 'No_Pol',
       'Car_Category', 'Subject_Car_Make', 'LGA_Name', 'State', 'ProductName',
       'source'],
      dtype='object')

In [83]:
df.Subject_Car_Make.value_counts()

TOYOTA        5466
Honda         1146
Lexus          654
Mercedes       577
Hyundai        512
              ... 
Datsun           1
REXTON           1
Yamaha           1
Buik             1
Rols Royce       1
Name: Subject_Car_Make, Length: 75, dtype: int64

In [84]:
####  Extracting columns from the date columns

In [85]:
date_columns = ['Policy Start Date','Policy End Date']
# creating a function to extract the year, day, month and quarter
def extract_date_info(df,cols,):
    for feat in cols:
        df[feat +'_year'] = df[feat].dt.year
        df[feat +'_day'] = df[feat].dt.day
        df[feat +'_month'] = df[feat].dt.month
        df[feat +'_quarter'] = df[feat].dt.quarter
    df.drop(columns=date_columns,axis=1,inplace=True)
    
extract_date_info(df,date_columns)

In [86]:
#### Replacing the some categories in the gender column with null values

In [87]:
df['Gender'] =df['Gender'].replace({"NO GENDER": "NOT STATED",
                                                   "SEX": np.NaN,"NO GENDER": np.NaN,'Joint Gender': np.NaN,'Entity':np.NaN})
df['Gender'].fillna(np.NaN, inplace=True)


In [88]:
df['Policy End Date_year'].value_counts()

2011    13096
2010      177
2016        2
2041        2
2012        2
2034        1
2037        1
Name: Policy End Date_year, dtype: int64

In [89]:
#### Noticed outliers in df['Policy End Date_year'] column and replacing the outliers with null values

In [90]:
df['Policy End Date_year'] = np.where(df['Policy End Date_year'].between(2015,2041), np.NaN, df['Policy End Date_year'])

In [91]:
## Checking the state columns
df.State.unique()

array([nan, 'Lagos', 'Benue', 'Eti-Osa', 'Delta', 'Ogun',
       'Abuja-Municipal', 'Oshodi-Isolo', 'Ibeju-Lekki', 'Ibadan-West',
       'Obia-Akpor', 'Ibadan-East', 'Kaduna-South', 'Amuwo-Odofin',
       'Anambra', 'Rivers', 'Niger-State', 'Edo', 'Akwa-Ibom',
       'ENUGU-EAST', 'Abuja', 'Kaduna', 'Esan-West', 'Obafemi-Owode',
       'Orile-Iganmu', 'Ifako-Ijaye', 'Ibadan-North', 'Imo', 'Jos-North',
       'Ondo-West', 'Kaduna-North', 'Aniocha-South', 'Awka-South',
       'Ile-Oluji', 'Ijebu-Ode', 'Port-Harcourt', 'Nnewi-North',
       'Aboh-Mbaise', 'Oyo', 'Akoko-West', 'Warri-Central', 'Cross-River',
       'Jos-South', 'Onitsha-North', 'Osun', 'Ajeromi-Ifelodun',
       'Kano-Municipal', 'Ife-Central', 'Ilorin-West', 'Lagelu-North',
       'Owerri-Municipal', 'Nsit-Ubium', 'Udi-Agwu', 'Essien-Udim',
       'Owerri-West', 'Ondo', 'Ogbmosho-South', 'Umuahia-South',
       'ABULE-EGBA', 'Enugu-North', 'Ovia-SouthWest', 'Kebbi',
       'Ajegunle-State', 'Ekiti-West', 'Ogun-Waterside',

In [92]:
## Replacing the states with the corrected details to reduce the reduce the number of categorical features and for a cleaner cstegorical column
df['State'] = df['State'].replace({'Abuja-Municipal':'Federal Capital Territory','Oshodi-Isolo':'Lagos','Ibeju-Lekki':'Lagos','Ibadan-West':'Ibadan','Eti-Osa':'Lagos'
                                        ,'Obia-Akpor':'Awka-Ibom','Ibadan-East':'Oyo','Amuwo-Odofin':'Lagos','ENUGU-EAST':'Enugu','Esan-West':'Edo'
                                         ,'Obafemi-Owode':'Ogun','Orile-Iganmu':'Lagos', 'Ifako-Ijaye':'Lagos', 'Ibadan-North':'Oyo',
                                         'Jos-North':'Plateau'
                                        , 'Ondo-West':'Ondo', 'Kaduna-North':'Kaduna','Aniocha-South':'Delta','Ile-Oluji':'Ondo','Nnewi-North':'Anambra'
                                        ,'Aboh-Mbaise':'Imo', 'Akoko-West':'Ondo','Warri-Central':'Edo', 'Jos-South':'Plateus','Onitsha-North':'Anambra'
                                        , 'Ajeromi-Ifelodun':'Osun', 'Kano-Municipal':'Kano','Ife-Central':'Osun','Ilorin-West':'Kwara'
                                         ,'Lagelu-North':'Oyo', 'Owerri-Municipal':'Imo','Nsit-Ubium':'Awka-Ibom', 'Udi-Agwu':'Enugu',
                                         'Owerri-West':'Imo'
                                         ,'Ogbmosho-South':'Oyo', 'Umuahia-South':'Abia','Ajegunle-State':'Lagos','Ekiti-West':'Ekiti',
                                         'Ogun-Waterside':'Ogun'
                                        ,'Awka-North':'Anambra','ABULE-EGBA':'Lagos','Ife-North':'Osun', 'Anambra-East':'Anambra', 'Etsako-West':'Edo'
                                         ,'Aba-North':'Abia','Ibarapa-Central':'Oyo', 'Idemili-North':'Anambra', 'Ogba-Ndoni':'Rivers', 
                                         'Calabar-Municipality':'Cross-River','AJAO-ESTATE':'Lagos','Ekiti-East':'Ekiti',
                                         'Ovia-SouthWest':'Edo','Essien-Udim':'Akwa-Ibom',
                                         'Ado-Ota':'Ogun', 'Oyo-East':'Oyo', 'Ilorin-East':'Kwara', 'Ilesha-West':'Osun', 'Central-Abuja':'Federal Capital Territory',
                                         'Owerri-North':'Imo','Ilesha-East':'Osun', 'Awka-South':'Delta','Enugu-North':'Enugu','Ibadan':'Oyo',
                                         'Esan-Central':'Edo', 'Isoko-south':'Delta', 'Ethiope-East':'Delta','Ijebu-Ode':'Ogun'
                                         ,'QuaAn-Pan':'Plateus', 'Warri-North':'Delta', 'Ado-Ekiti':'Ekiti', 'Ijebu-East':'Ogun',
                                         'Onitsha-South':'Anambra', 'Ughelli-North':'Delta', 'Warri-South':'Delta', 'Kaduna-South':'Kaduna',
                                         'Port-Harcourt':'Rivers', 'N-A':np.NaN, 'Idemili-south': 'Anambra', 'Ngor-Okpala':'Imo',
      
                                         'Oyo-West':'Oyo', 'Aba-South':'Abia', 'Isoko-North':'Delta', 'Oshimili-North':'Delta',
                                         'Ndokwa-East':'Delta', 'Nnewi-South':'Anambra', 'ENUGU-SOUTH':'Enugu', 'Ijebu-North':'Ogun',
                                         'Asari-Toru':'Rivers', 'Akwa-Ibom':'Awka-Ibom'})
df.State.fillna(np.NaN,inplace =True)

df['LGA_Name']=df.LGA_Name.replace({'LGA':np.NaN,'Victoria Island':'Eti-Osa','Ikoyi':'Eti-Osa','Yaba':'Lagos Mainland'
                                         ,'GBAGADA':'Kosofe','ISHERI':'Kosofe','OBANIKORO':'Shomolu','Ebute-Metta':'Lagos Mainland'
                                         ,'Lekki':'Ibeju/Lekki', 'Festac':'Amuwo-Odofin', 'Ipaja':'Alimosho','Egbeda':'Ikeja',
                                          'Orile-Iganmu':'Surulere','Alagbado':'Alimosho',
                                          'Ajegunle,Lagos  State':'Ifako-Ijaiye', 'Dopemu':'Agege', 
                                           'Akoka':'Shomolu','Ijora':'Apapa', ' IFAKO':'Ifako-Ijaiye'
                                         ,'Ejigbo':'Oshodi-Isolo','AJAO ESTATE':'Oshodi-Isolo', 'Ikotun':'Alimosho',
                                          'Obalende':'Eti-Osa', 'Central Area, Abuja':'Abuja','Central Area':'Abuja','Central':'Abuja'
                                           ,'BARIGA':'Shomolu', 'ONIRU':'Eti-Osa', 'MAGBORO':'Obafemi-Owode','ALAPERE':'Kosofe',
                                          'Palm Groove':'Shomolu','Somolu':'Shomolu','OGBA':'Ifako-Ijaiye'
                                         ,'Lagos':'Lagos Mainland', 'Ajah':'Eti-Osa','Lagelu Ogbomosho North':'Ogbomosho North',
                                         'Ilupeju':'Mushin', 'Anthony Village':'Kosofe', 'Isolo':'Oshodi-Isolo','MAGODO':'Ikeja','Ile-Ife':'Ife Central',
                                          'Maryland':'Kosofe','Oshodi':'Oshodi-Isolo','Ketu':'Kosofe','Aguda':'Surulere','ABULE EGBA':'Alimosho',
                                          'OJOTA':'Kosofe','Ifako-Ijaye ':'Ifako-Ijaiye','Ikeja G.R.A':'Ikeja','OWODE':'Obafemi-Owode',
                                          'Ifako-Ijaye':'Ifako-Ijaiye', 'Ogbmosho South': 'Ogbomosho South','OGUDU':'Ogudu',
                                          'ENUGU EAST':'Enugu East','RIVERS':'Rivers','AKOKO-EDO':'Akoko-Edo','Shomulu':'Shomolu'
                                          ,'ENUGU SOUTH':'Enugu South','Abuja':'Abuja Municipal', 'OGUN':'Abeokuta','Niger State':'Minna',
                                          'Oyo':'IbadanCentral','IBADAN NORTH EAST':'Ibadan North East','Rivers':'Port-Harcourt', 'Akwa Ibom':'Uyo',
                                          'SANGOTEDO':'Eti-Osa','EMENE':'Enugu East','EFFURUN':'Uvwie','Asaba': 'Oshimili-North', 'Awka':'Awka South',
                                          'Minna':'Chanchaga','Oredo':'Benin','Warri':'Warri South','ONDO':'Akure','Wuse 11':'Abuja Municipal',
                                          'AKUTE':'Ifo','Ojodu':'Ikeja','Victoria Island':'Eti-Osa','Ikoyi':'Eti-Osa','Yaba':'Lagos Mainland'
                                         ,'GBAGADA':'Kosofe','ISHERI':'Kosofe','OBANIKORO':'Shomolu','Ebute-Metta':'Lagos Mainland'
                                         ,'Lekki':'Ibeju/Lekki', 'Festac':'Amuwo-Odofin', 'Ipaja':'Alimosho','Egbeda':'Ikeja',
                                          'Orile-Iganmu':'Surulere','Alagbado':'Alimosho',
                                          'Ajegunle,Lagos  State':'Ifako-Ijaiye', 'Dopemu':'Agege', 
                                           'Akoka':'Shomolu','Ijora':'Apapa', ' IFAKO':'Ifako-Ijaiye'
                                         ,'Ejigbo':'Oshodi-Isolo','AJAO ESTATE':'Oshodi-Isolo', 'Ikotun':'Alimosho',
                                          'Obalende':'Eti-Osa', 'Central Area, Abuja':'Abuja','Central Area':'Abuja','Central':'Abuja'
                                           ,'BARIGA':'Shomolu', 'ONIRU':'Eti-Osa', 'MAGBORO':'Obafemi-Owode','ALAPERE':'Kosofe',
                                          'Palm Groove':'Shomolu','Somolu':'Shomolu','OGBA':'Ifako-Ijaiye'
                                         ,'Lagos':'Lagos Mainland', 'Ajah':'Eti-Osa','Lagelu Ogbomosho North':'Ogbomosho North',
                                         'Ilupeju':'Mushin', 'Anthony Village':'Kosofe', 'Isolo':'Oshodi-Isolo','MAGODO':'Ikeja','Ile-Ife':'Ife Central',
                                          'Maryland':'Kosofe','Oshodi':'Oshodi-Isolo','Ketu':'Kosofe','Aguda':'Surulere','ABULE EGBA':'Alimosho',
                                          'OJOTA':'Kosofe','Ifako-Ijaye ':'Ifako-Ijaiye','Ikeja G.R.A':'Ikeja','OWODE':'Obafemi-Owode',
                                          'Ifako-Ijaye':'Ifako-Ijaiye', 'Ogbmosho South': 'Ogbomosho South','OGUDU':'Ogudu',
                                          'ENUGU EAST':'Enugu East','RIVERS':'Rivers','AKOKO-EDO':'Akoko-Edo','Shomulu':'Shomolu'
                                          ,'ENUGU SOUTH':'Enugu South','Abuja':'Abuja Municipal', 'OGUN':'Abeokuta','Niger State':'Minna',
                                          'Oyo':'IbadanCentral','IBADAN NORTH EAST':'Ibadan North East','Rivers':'Port-Harcourt', 'Akwa Ibom':'Uyo',
                                          'SANGOTEDO':'Eti-Osa','EMENE':'Enugu East','EFFURUN':'Uvwie','Asaba': 'Oshimili-North', 'Awka':'Awka South',
                                          'Minna':'Chanchaga','Oredo':'Benin','Warri':'Warri South','ONDO':'Akure','Wuse 11':'Abuja Municipal',
                                          'AKUTE':'Ifo','Ojodu':'Ikeja'
                                         })
df.LGA_Name.fillna(np.NaN,inplace =True)

In [93]:
df['LGA_Name']=df.LGA_Name.replace({'LGA':np.NaN})
df.LGA_Name.fillna(np.NaN,inplace =True)

In [94]:
df.isnull().sum()/len(train)* 100

Gender                        8.742446
Age                           5.886249
No_Pol                        0.000000
Car_Category                 34.025996
Subject_Car_Make             22.551536
LGA_Name                     59.152248
State                        59.152248
ProductName                   0.000000
source                        0.000000
Policy Start Date_year        0.000000
Policy Start Date_day         0.000000
Policy Start Date_month       0.000000
Policy Start Date_quarter     0.000000
Policy End Date_year          0.049673
Policy End Date_day           0.000000
Policy End Date_month         0.000000
Policy End Date_quarter       0.000000
dtype: float64

In [95]:
### Replacing the null values with their mode

In [96]:
cat_cols=['State','ProductName','Car_Category','Subject_Car_Make','Gender','LGA_Name']
for i in cat_cols:
    df[i] = df[i].fillna(df[i].mode()[0])

In [97]:
### Saving the categorical columns as type category
df[cat_cols] = df[cat_cols].astype('category')

In [98]:
## Splitting of the columns back to train and test and dropping the indicator column
train = df.loc[df['source']=='train']
test = df.loc[df['source']=='test']

train.drop('source',axis=1,inplace=True)
test.drop('source',axis =1,inplace =True)

In [99]:
train.head(2)

,Gender,Age,No_Pol,Car_Category,Subject_Car_Make,LGA_Name,State,ProductName,Policy Start Date_year,Policy Start Date_day,Policy Start Date_month,Policy Start Date_quarter,Policy End Date_year,Policy End Date_day,Policy End Date_month,Policy End Date_quarter
0,Male,30.0,1,Saloon,TOYOTA,Victoria Island,Lagos,Car Classic,2010,14,5,2,2011.0,13,5,2
1,Female,79.0,1,JEEP,TOYOTA,Victoria Island,Lagos,Car Classic,2010,29,11,4,2011.0,28,11,4


In [100]:
test.head(2)

,Gender,Age,No_Pol,Car_Category,Subject_Car_Make,LGA_Name,State,ProductName,Policy Start Date_year,Policy Start Date_day,Policy Start Date_month,Policy Start Date_quarter,Policy End Date_year,Policy End Date_day,Policy End Date_month,Policy End Date_quarter
12079,Female,46.0,1,Saloon,Ford,Abuja Municipal,Federal Capital Territory,Car Classic,2010,23,10,4,2011.0,22,10,4
12080,Male,32.0,1,Saloon,TOYOTA,Kosofe,Benue,Car Classic,2010,14,10,4,2011.0,13,10,4


In [101]:
def inverser(df,cols):
    df_ = df.copy()
    for i in cols:
        df_[i] = df_[i].replace(MLE_._display(i))
    return df_

In [102]:
MLE_ = MLETransformer(cat_cols)
train = MLE_.fit_transform(train)

In [103]:
train.head(2)

,Gender,Age,No_Pol,Car_Category,Subject_Car_Make,LGA_Name,State,ProductName,Policy Start Date_year,Policy Start Date_day,Policy Start Date_month,Policy Start Date_quarter,Policy End Date_year,Policy End Date_day,Policy End Date_month,Policy End Date_quarter
0,1,30.0,1,8,67,246,23,1,2010,14,5,2,2011.0,13,5,2
1,0,79.0,1,2,67,246,23,1,2010,29,11,4,2011.0,28,11,4


In [104]:
test.head(2)

,Gender,Age,No_Pol,Car_Category,Subject_Car_Make,LGA_Name,State,ProductName,Policy Start Date_year,Policy Start Date_day,Policy Start Date_month,Policy Start Date_quarter,Policy End Date_year,Policy End Date_day,Policy End Date_month,Policy End Date_quarter
12079,Female,46.0,1,Saloon,Ford,Abuja Municipal,Federal Capital Territory,Car Classic,2010,23,10,4,2011.0,22,10,4
12080,Male,32.0,1,Saloon,TOYOTA,Kosofe,Benue,Car Classic,2010,14,10,4,2011.0,13,10,4


In [105]:
# ff.head()

In [106]:
# test['LGA_Name'].value_counts()

In [107]:
MLE_._display('Subject_Car_Make')

{67: 'TOYOTA',
 57: 'REXTON',
 42: 'Lexus',
 29: 'Hyundai',
 33: 'Iveco',
 17: 'DAF',
 26: 'Honda',
 49: 'Mercedes',
 36: 'Jincheng',
 2: 'ACURA',
 21: 'Ford',
 69: 'Volkswagen',
 52: 'Nissan',
 0: '.',
 55: 'Pontiac',
 58: 'Range Rover',
 38: 'Kia',
 50: 'Mitsubishi',
 62: 'Scania',
 7: 'BMW',
 30: 'Infiniti',
 60: 'Renault',
 70: 'Volvo',
 28: 'Hummer',
 46: 'Mack',
 25: 'Grand Cherokee',
 56: 'Porsche',
 54: 'Peugeot',
 40: 'Land Rover',
 15: 'Chevrolet',
 48: 'Mazda',
 47: 'Man',
 35: 'Jeep',
 6: 'Audi',
 66: 'Suzuki',
 45: 'MINI COOPER',
 37: 'KA',
 3: 'As Attached',
 31: 'Innson',
 32: 'Isuzu',
 64: 'Skoda',
 34: 'Jaguar',
 16: 'Chrysler',
 18: 'Dodge',
 23: 'GMC',
 41: 'Land Rover.',
 65: 'Subaru',
 22: 'GAC',
 20: 'Fiat',
 53: 'Opel',
 5: 'Astra',
 51: 'Motorcycle',
 73: 'ZOYTE',
 63: 'Seat',
 19: 'FOTON',
 27: 'Howo',
 43: 'Lincoln',
 61: 'Rols Royce',
 12: 'CHANGAN',
 39: 'LIBERTY',
 1: 'ABG',
 72: 'Yamaha',
 10: 'Black',
 68: 'Tata',
 4: 'Ashok Leyland',
 24: 'Geely',
 14: '

In [108]:
train.columns

Index(['Gender', 'Age', 'No_Pol', 'Car_Category', 'Subject_Car_Make',
       'LGA_Name', 'State', 'ProductName', 'Policy Start Date_year',
       'Policy Start Date_day', 'Policy Start Date_month',
       'Policy Start Date_quarter', 'Policy End Date_year',
       'Policy End Date_day', 'Policy End Date_month',
       'Policy End Date_quarter'],
      dtype='object')

In [109]:
# train.isnull().sum()

In [110]:
ff = train.copy()

In [111]:
train['Car_Category'].value_counts()

8     9772
2     2021
13      97
0       54
3       36
6       30
5       17
4       13
9       12
15      10
10       6
11       5
14       3
1        1
12       1
7        1
Name: Car_Category, dtype: int64

In [112]:
inverser(ff,cat_cols)['Car_Category'].value_counts()

Saloon                     9772
JEEP                       2021
Truck                        97
Bus                          54
Mini Bus                     36
Pick Up                      30
Motorcycle                   17
Mini Van                     13
Sedan                        12
Wagon                        10
Shape Of Vehicle Chasis       6
Station 4 Wheel               5
Van                           3
Tipper Truck                  1
Pick Up > 3 Tons              1
CAMRY CAR HIRE                1
Name: Car_Category, dtype: int64

In [113]:
ff.head()

,Gender,Age,No_Pol,Car_Category,Subject_Car_Make,LGA_Name,State,ProductName,Policy Start Date_year,Policy Start Date_day,Policy Start Date_month,Policy Start Date_quarter,Policy End Date_year,Policy End Date_day,Policy End Date_month,Policy End Date_quarter
0,1,30.0,1,8,67,246,23,1,2010,14,5,2,2011.0,13,5,2
1,0,79.0,1,2,67,246,23,1,2010,29,11,4,2011.0,28,11,4
2,1,43.0,1,8,67,246,23,1,2010,21,3,1,2011.0,20,3,1
3,1,2.0,1,8,67,246,23,5,2010,21,8,3,2011.0,20,8,3
4,1,20.0,3,8,67,156,23,8,2010,29,8,3,2010.0,31,12,4


In [114]:
train.isnull().sum()

Gender                         0
Age                          642
No_Pol                         0
Car_Category                   0
Subject_Car_Make               0
LGA_Name                       0
State                          0
ProductName                    0
Policy Start Date_year         0
Policy Start Date_day          0
Policy Start Date_month        0
Policy Start Date_quarter      0
Policy End Date_year           6
Policy End Date_day            0
Policy End Date_month          0
Policy End Date_quarter        0
dtype: int64

In [115]:
train['Age'].fillna(18 , inplace=True)

train.fillna(0 , inplace=True)

# Create synthetic samples
sm = SMOTE(random_state=2021, sampling_strategy='minority') ##NearMiss(version=3, n_neighbors_ver3=3)#
train, y = sm.fit_resample(train, y)

print(y.value_counts())

0    10624
1    10624
Name: target, dtype: int64


In [116]:
train.shape, y.shape

((21248, 16), (21248,))

In [117]:
ff = train.copy()
inverser(ff,cat_cols)['Car_Category'].value_counts()

Saloon                     16308
JEEP                        2937
Pick Up                      322
Pick Up > 3 Tons             315
Mini Bus                     299
Mini Van                     297
Motorcycle                   287
Truck                        130
Bus                           80
Sedan                         68
Shape Of Vehicle Chasis       60
Tipper Truck                  55
Station 4 Wheel               42
CAMRY CAR HIRE                32
Wagon                         10
Van                            6
Name: Car_Category, dtype: int64

In [118]:
# train.head()

In [119]:
# df.head()

In [120]:
# train

In [121]:
train.head()

,Gender,Age,No_Pol,Car_Category,Subject_Car_Make,LGA_Name,State,ProductName,Policy Start Date_year,Policy Start Date_day,Policy Start Date_month,Policy Start Date_quarter,Policy End Date_year,Policy End Date_day,Policy End Date_month,Policy End Date_quarter
0,1,30.0,1,8,67,246,23,1,2010,14,5,2,2011.0,13,5,2
1,0,79.0,1,2,67,246,23,1,2010,29,11,4,2011.0,28,11,4
2,1,43.0,1,8,67,246,23,1,2010,21,3,1,2011.0,20,3,1
3,1,2.0,1,8,67,246,23,5,2010,21,8,3,2011.0,20,8,3
4,1,20.0,3,8,67,156,23,8,2010,29,8,3,2010.0,31,12,4


In [122]:
test.head()

,Gender,Age,No_Pol,Car_Category,Subject_Car_Make,LGA_Name,State,ProductName,Policy Start Date_year,Policy Start Date_day,Policy Start Date_month,Policy Start Date_quarter,Policy End Date_year,Policy End Date_day,Policy End Date_month,Policy End Date_quarter
12079,Female,46.0,1,Saloon,Ford,Abuja Municipal,Federal Capital Territory,Car Classic,2010,23,10,4,2011.0,22,10,4
12080,Male,32.0,1,Saloon,TOYOTA,Kosofe,Benue,Car Classic,2010,14,10,4,2011.0,13,10,4
12081,Female,45.0,2,Saloon,Honda,Wuse 11,Abuja,Car Classic,2010,29,8,3,2011.0,28,8,3
12082,Female,58.0,1,Saloon,TOYOTA,Victoria Island,Lagos,CarSafe,2010,13,6,2,2011.0,12,6,2
12083,Male,NaN,1,Saloon,Hyundai,Victoria Island,Lagos,Car Classic,2010,1,7,3,2011.0,30,6,2


In [123]:
X = inverser(train,cat_cols) #train

test = MLE_.transform(test)
test_new = inverser(test,cat_cols)

In [136]:
test_new['Age'].fillna(18 , inplace=True)

test_new.isnull().sum()

Gender                       0
Age                          0
No_Pol                       0
Car_Category                 0
Subject_Car_Make             0
LGA_Name                     0
State                        0
ProductName                  0
Policy Start Date_year       0
Policy Start Date_day        0
Policy Start Date_month      0
Policy Start Date_quarter    0
Policy End Date_year         0
Policy End Date_day          0
Policy End Date_month        0
Policy End Date_quarter      0
dtype: int64

In [128]:
X.head()

,Gender,Age,No_Pol,Car_Category,Subject_Car_Make,LGA_Name,State,ProductName,Policy Start Date_year,Policy Start Date_day,Policy Start Date_month,Policy Start Date_quarter,Policy End Date_year,Policy End Date_day,Policy End Date_month,Policy End Date_quarter
0,Male,30.0,1,Saloon,TOYOTA,Victoria Island,Lagos,Car Classic,2010,14,5,2,2011.0,13,5,2
1,Female,79.0,1,JEEP,TOYOTA,Victoria Island,Lagos,Car Classic,2010,29,11,4,2011.0,28,11,4
2,Male,43.0,1,Saloon,TOYOTA,Victoria Island,Lagos,Car Classic,2010,21,3,1,2011.0,20,3,1
3,Male,2.0,1,Saloon,TOYOTA,Victoria Island,Lagos,CarSafe,2010,21,8,3,2011.0,20,8,3
4,Male,20.0,3,Saloon,TOYOTA,Lagos,Lagos,Muuve,2010,29,8,3,2010.0,31,12,4


In [137]:
test_new.head()

,Gender,Age,No_Pol,Car_Category,Subject_Car_Make,LGA_Name,State,ProductName,Policy Start Date_year,Policy Start Date_day,Policy Start Date_month,Policy Start Date_quarter,Policy End Date_year,Policy End Date_day,Policy End Date_month,Policy End Date_quarter
12079,Female,46.0,1,Saloon,Ford,Abuja Municipal,Federal Capital Territory,Car Classic,2010,23,10,4,2011.0,22,10,4
12080,Male,32.0,1,Saloon,TOYOTA,Kosofe,Benue,Car Classic,2010,14,10,4,2011.0,13,10,4
12081,Female,45.0,2,Saloon,Honda,Wuse 11,Abuja,Car Classic,2010,29,8,3,2011.0,28,8,3
12082,Female,58.0,1,Saloon,TOYOTA,Victoria Island,Lagos,CarSafe,2010,13,6,2,2011.0,12,6,2
12083,Male,18.0,1,Saloon,Hyundai,Victoria Island,Lagos,Car Classic,2010,1,7,3,2011.0,30,6,2


In [127]:
# X['State'] = X['State'].replace(MLE_._display('State'))
# X['Subject_Car_Make'] = X['Subject_Car_Make'].replace(MLE_._display('Subject_Car_Make'))
# X['LGA_Name'] = X['LGA_Name'].replace(MLE_._display('LGA_Name'))

In [49]:
# MLE_._display('LGA_Name')

In [50]:
#  X['LGA_Name'].replace(MLE_._display('LGA_Name'))

In [51]:
## Cross validation and model selection

In [129]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold,StratifiedShuffleSplit

In [130]:
import lightgbm
lightgbm.__version__

'3.1.1'

In [ ]:
# !pip install MultiLabel-Transformer

In [256]:
test_new[cat_cols].head(2)

,State,ProductName,Car_Category,Subject_Car_Make,Gender,LGA_Name
12079,Federal Capital Territory,Car Classic,Saloon,Ford,Female,Abuja Municipal
12080,Benue,Car Classic,Saloon,TOYOTA,Male,Kosofe


In [139]:
### Saving the categorical columns as type category
X[cat_cols] = X[cat_cols].astype('category')

### Saving the categorical columns as type category
test_new[cat_cols] = test_new[cat_cols].astype('category')

In [260]:
seed = 2021 # seed
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed) # for cross validation
scores = []
preds= []


#creating a for loop for the stratified k fold
i = 1
for train, test in skf.split(X, y):
    x_train, x_test, y_train, y_test = X.iloc[train], X.iloc[test], y.iloc[train], y.iloc[test]# spliting the data
    estimator = LGBMClassifier(n_jobs=-1 ,learning_rate=0.123, n_estimators = 50, deterministic=True,
#                                reg_alpha = 0.01  ,#feature_fraction=0.9 , #min_sum_hessian_in_leaf=1e-51,
                               scale_pos_weight= 1 , max_depth=16, num_leaves=50,reg_lambda=0.3)# parameters for the algorithm this was done by manual tuning

    estimator.fit( x_train, y_train )# fitting on train data
    
    print('Number of splits trained {} '.format(i))
    score = f1_score(y_test, estimator.predict(x_test)) # checking the accuracy
    pred = estimator.predict_proba(test_new)[:,1] # making prediction probabities
    scores.append(score)
    preds.append(pred)
    i += 1

np.mean(scores)

Number of splits trained 1 
Number of splits trained 2 
Number of splits trained 3 
Number of splits trained 4 
Number of splits trained 5 
Number of splits trained 6 
Number of splits trained 7 
Number of splits trained 8 
Number of splits trained 9 
Number of splits trained 10 


0.8804867682858408

In [245]:
pred

array([0.19859368, 0.32206247, 0.07172491, ..., 0.43349807, 0.1444112 ,
       0.70976928])

In [246]:
preds_mean = np.mean(preds, axis=0)# mean of all the predictions

In [247]:
preds_mean.shape # shape of the mean predictions

(1202,)

In [251]:
## fro the final score since predict probalities was used, a threshold of 0.5 was used to group the preditions into 0 and 1 respectively
final = []
for x in preds_mean:
    if x >= 0.53:
        final.append(1)
    else:
        final.append(0)
submit = ss.copy() # a copy of the submission sample csv
submit.target = final
submit.to_csv('XLB_.csv',index=False) # naming the file

In [252]:
import joblib

model_name = 'lgbm_model.joblib' 
joblib.dump(estimator , model_name)

['lgbm_model.joblib']

In [253]:
submit.target.value_counts()# value count of the target column

0    1067
1     135
Name: target, dtype: int64

##### 